# 네이버 플레이스 블로그 리뷰 크롤링
#### 2020-05-10 (일) 이한울
---
- 2020-05-10 (일): 플레이스 블로그 리뷰 긁어오기 및 스레딩 적용 완료
- 2020-05-11 (월): 스레딩 실행 창에서 driver 경로를 본인의 크롬 드라이버가 있는 경로로 설정 후 실행
- 2020-05-11 (월): time.sleep을 넉넉하게 주어야 HTTP 에러를 최대한 회피할 수 있습니다.

In [14]:
import  urllib.request
import selenium
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

from pandas import Series, DataFrame
import pandas as pd
import numpy as np

from datetime import datetime
import time
import sys
import math
import re
from tqdm import tqdm, tqdm_notebook

import selenium.common.exceptions as sce
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

from queue import Queue
from multiprocessing.pool import ThreadPool
from threading import Thread
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

import glob
import fnmatch

In [15]:
# 블로그 검색 결과를 저장할 빈 DataFrame 생성
df_blog_review = DataFrame(columns=['store_id', 'store_name', 'title', 'writer', 'link', 'summary', 'blog_name', 'total_number'])

In [16]:
df_blog_error = DataFrame(columns=['store_id','store_name','error_code', 'original_index'])

In [17]:
df_blog_empty = DataFrame(columns=['store_id','store_name','original_index','blog_tab'])

In [18]:
# 블로그 본문을 저장할 빈 DataFrame 생성
df_text = DataFrame(columns=['store_id', 'store_name', 'link', 'main text'])

In [19]:
# 플레이스의 업소별 ID를 담은 csv파일 DataFrame으로 변환
df_place = DataFrame(pd.read_csv('./naver-place_csv/naver_place_count_total_2020-05-01_final.csv', sep=',', header=0))

In [20]:
# Unnamed column drop
df_place.drop('Unnamed: 0', axis=1, inplace=True)
df_place

,store_id,store_id_only_num,store_name,booking,receipt,blog,total
0,s1518997092,1518997092,준오헤어 노원사거리점,2015.0,3.0,15.0,2033.0
1,s1571970153,1571970153,차홍룸 홍대점,2009.0,1.0,10.0,2020.0
2,s1142939643,1142939643,차홍룸 용산센트럴점,1871.0,3.0,11.0,1885.0
3,s1481443590,1481443590,차홍룸 용산점,1541.0,2.0,11.0,1554.0
4,s1177346002,1177346002,차홍룸 양재점,1373.0,1.0,10.0,1384.0
...,...,...,...,...,...,...,...
2722,s38257322,38257322,바바헤어 대치점,1.0,1.0,2.0,4.0
2723,s1069634711,1069634711,정환헤어샵,1.0,1.0,2.0,4.0
2724,s20852762,20852762,머리짱헤어샵,1.0,2.0,1.0,4.0
2725,s36091089,36091089,살롱160,1.0,1.0,2.0,4.0


# 네이버 플레이스 모바일 페이지 크롤링 함수
#### 2020-05-12 (화) 이한울
---
- 스레딩 테스트 중

In [21]:
# status가 400, 500인 경우 크롤링하지 못한 업소의 정보를 따로 저장하는 함수
def df_error_save(store_id, store_name, error_code):
    
    global df_blog_error
    global df_place
    
    original_index = \
    df_place[df_place['store_id_only_num'] == store_id]['store_name'].index
    
    df_blog_error = df_blog_error.append({
        'store_id' : store_id
        , 'store_name' : store_name
        , 'error_code' : error_code
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_blog_error

In [22]:
# 크롤링이 완료된 업소의 정보들을 DataFrame에 저장하는 함수
def df_info_save(store_id, store_name, title, blog_writer, link, summary, blog_name, total_number):
    
    global df_blog_review
    
    df_blog_review = df_blog_review.append({
        'store_id' : store_id
        , 'store_name' : store_name
        , 'title' : title
        , 'writer' : blog_writer
        , 'link' : link
        , 'summary' : summary
        , 'blog_name' : blog_name
        , 'total_number' : 101 # total_number는 바꿔야함
    }, ignore_index=True)
    
    return df_blog_review

In [23]:
# 플레이스 검색 결과 블로그 리뷰가 하나도 존재하지 않는 업소 정보 저장 함수
def df_no_result(store_id, store_name, blog_tab):
    
    global df_blog_empty
    global df_place
    
    original_index = \
    df_place[df_place['store_id_only_num'] == store_id]['store_name'].index
    
    df_blog_empty = df_blog_empty.append({
        'store_id' : store_id
        , 'store_name' : store_name
        , 'blog_tab' : ('https://m.place.naver.com' + blog_tab)
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_empty

In [24]:
def m_place_save(driver, store):
    
    try:
        count_list = []
        store_id = store
        total_number = 1
    #     driver = Chrome('./chromedriver.exe')
        store_name = str(df_place[df_place['store_id_only_num'] == store_id]['store_name'].iloc[0])

        driver.get('https://m.store.naver.com/hairshops/detail?entry=plt&id=' + str(store_id)) # df_blog['links'][i]
        status_req = requests.get('https://m.store.naver.com/hairshops/detail?entry=plt&id=' + str(store_id))
        time.sleep(2)

        if status_req.status_code in (500, 501, 400, 403, 404, 401, 503):

            print(store, store_name, '크롤링 중 에러가 발생했습니다.')
            print('내부 서버 에러가 발생했습니다.')
            print('30초 후에 다시 진행합니다.')
            df_error_save(store, store_name, status_req.status_code)

        elif status_req.status_code == 200:

            time.sleep(1.5 + (np.random.rand()))
            place_soup = BeautifulSoup(driver.page_source, 'html.parser')
            span = place_soup.find_all('span', '_1Y6hi')[-1]
            link = (span.a).attrs['href']

            if '/review/ugc' in link:

                print(store, store_name, '크롤링을 진행합니다.\n')
                blog_tab = link
                total_count = (((span.a).get_text()).strip()).replace('블로그리뷰 ', '')
                print('명시적 총 블로그 리뷰 수:', int(total_count), '\n')
                print('블로그 탭:', blog_tab, '\n')
                time.sleep(2)
                driver.get('https://m.place.naver.com' + blog_tab)
                req = requests.get('https://m.store.naver.com' + blog_tab)

                time.sleep(1.5 + (np.random.rand() + np.random.rand()))

                if int(total_count) > 10:
                    # 블로그 리뷰 페이지의 다음 버튼 클릭
                    blog_x_path = '//a[@class="_3iTUo"]'        

                    if driver.find_element(By.XPATH, blog_x_path).is_displayed():

                        try:
                            while driver.find_element(By.XPATH, blog_x_path).is_displayed():

                                next_btn = driver.find_element(By.XPATH, blog_x_path)
                                next_btn.click()
                                time.sleep(2)

                        except sce.NoSuchElementException:
                            print('해당 엘리먼트를 찾지 못했습니다.')

                        except sce.ElementNotInteractableException:
                            print('해당 엘리먼트를 찾던 중 에러가 발생했습니다.')

                        time.sleep(2.5)
                        review_soup = BeautifulSoup(driver.page_source, 'html.parser')

                        # 첫 번째 페이지의 블로그 이름, 포스트 제목, 링크, 본문 요약 가져오기  
                        blog_src = ((review_soup.find('div', class_='place_section undefined')).find('ul', class_='_1fYfG undefined')).find_all('li', class_="_3xK_0")
                        
                        # 파싱 후 접근하고자 하는 요소가 존재할 때
                        if blog_src != None:
                            for blog in tqdm_notebook(blog_src):

                                link = (blog.a).attrs['href']
                                summary = (blog.find_all('div', class_="_3kKqj"))[1].get_text()
                                title = blog.find('div', '_2dj8M').get_text()
                                blog_name = blog.find('span', class_='_1vuEi').get_text()

                                # 블로그 작성자의 ID를 가져오기 위해 블로그 링크 분할
                                split_link = re.split('[/+]|[.+]|[?]+', str(link))

                                # 도메인이 blog.naver.com인 경우 블로그 작성자 가져오기
                                if split_link[3] == 'blog':
                                    blog_writer = split_link[6]    

                                else:
                                    pass

                                if ('cafeRedirect' or 'post.naver.com') not in link:
                                    count_list.append(link)
                                    df_info_save(store_id
                                                 , store_name
                                                 , title
                                                 , blog_writer
                                                 , link
                                                 , summary
                                                 , blog_name
                                                 , total_number)

                        # 파싱 후 접근하고자 하는 요소가 존재하지 않을 때
                        else:
                            print('파싱한 HTML 소스가 존재하지 않습니다!')
                            df_no_result(store, store_name, blog_tab)
                        
                else:
                    time.sleep(2.5)
                    review_soup = BeautifulSoup(driver.page_source, 'html.parser')

                    # 첫 번째 페이지의 블로그 이름, 포스트 제목, 링크, 본문 요약 가져오기  
                    blog_src = ((review_soup.find('div', class_='place_section undefined')).find('ul', class_='_1fYfG undefined')).find_all('li', class_="_3xK_0")
                    blog_x_path = None

                    for blog in tqdm_notebook(blog_src):

                        link = (blog.a).attrs['href']
                        summary = (blog.find_all('div', class_="_3kKqj"))[1].get_text()
                        title = blog.find('div', '_2dj8M').get_text()
                        blog_name = blog.find('span', class_='_1vuEi').get_text()

                        # 블로그 작성자의 ID를 가져오기 위해 블로그 링크 분할
                        split_link = re.split('[/+]|[.+]|[?]+', str(link))

                        # 도메인이 blog.naver.com인 경우 블로그 작성자 가져오기
                        if split_link[3] == 'blog':
                            blog_writer = split_link[6]

                        else:
                            pass

                        if ('cafeRedirect' or 'post.naver.com') not in link:
                            count_list.append(link)
                            df_info_save(store_id
                                         , store_name
                                         , title
                                         , blog_writer
                                         , link
                                         , summary
                                         , blog_name
                                         , total_number)

                    print('다음 페이지가 없습니다.')
                    pass

            else:
                print('블로그 리뷰가 없습니다.')
                pass
            
            if len(count_list) == 0:
                print('저장된 블로그 정보가 없습니다.')
                df_no_result(store, store_name, blog_tab)
            
    except sce.NoSuchElementException:
        print('블로그 리뷰가 없습니다!')
        print('다음 업소의 크롤링을 진행합니다.')
        df_no_result(store, store_name, blog_tab)
        
    except sce.ElementNotInteractableException:
        print('해당 엘리먼트를 찾던 중 에러가 발생했습니다.')
    
    print(store_name + '의 블로그 리뷰 총 개수:', len(count_list))
    driver.close()
    driver.quit()
    return df_blog_review

In [25]:
# 크롬 웹 드라이버 헤드리스 설정 및 웹 보안 무시
options = webdriver.ChromeOptions()

prefs = {
    'profile.default_content_setting_values': {
          'cookies': 2, 'images': 2, 'plugins': 2
        , 'popups': 2, 'geolocation': 2, 'notifications': 2
        , 'auto_select_certificate': 2, 'fullscreen': 2, 'mouselock': 2
        , 'mixed_script': 2, 'media_stream': 2, 'media_stream_mic': 2
        , 'media_stream_camera': 2, 'protocol_handlers': 2, 'ppapi_broker': 2
        , 'automatic_downloads': 2, 'midi_sysex': 2, 'push_messaging': 2
        , 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 'protected_media_identifier': 2
        , 'app_banner': 2, 'site_engagement': 2, 'durable_storage': 2
    }
}

options.add_experimental_option('prefs', prefs)
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('headless')
options.add_argument("disable-gpu")
options.add_argument("--disable-web-security")

In [27]:
# 함수의 스레딩 적용

# =========================================================================
# 1인당 대략 545개 링크
# A: start = 0, end = 544 → a: 0 
# df_place['blog'][:545].sum() = 5117

# B: start = 545, end = 1089 → a: 545 
# df_place['blog'][545:1090].sum() = 3773

# C: start = 1090, end = 1634 → a: 1090
# D: start = 1635, end = 2179 → a: 1635
# E: start = 2180, end = 2728 → a: 2180
end = 1089
# =========================================================================

# 2020-05-11 (월) 14:46 크롤링 시작
# 테스트로 2번만 돌려본 것이라서 range(0, 109)로 바꿔주어야 한다.
dlist = list(range(0, 109))
a = 545
for i in dlist:
    
    q = Queue(maxsize=5)
    for _ in range(q.maxsize):
        driver = Chrome('./chromedriver.exe', options=options)
        q.put(driver)
    
    start_no = a
    end_no = a+5
    print(str(start_no) + ' ~ ' + str((end_no) - 1) + '의 블로그 리뷰 크롤링을 시작합니다.')
    with ThreadPool(5) as tp:
        # 스레드에 입력할 매개변수가 2개 이상일 경우 starmap을 사용한다.
        tp.starmap(
            m_place_save
            , [(q.get(), df_place['store_id_only_num'][ii]) for ii in tqdm(range(start_no, end_no))]
        )
    a += 5
    time.sleep(3.5)
    if a >= (end - 1):
        break

0 ~ 4의 블로그 리뷰 크롤링을 시작합니다.


100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]


1142939643 차홍룸 용산센트럴점 크롤링을 진행합니다.
명시적 총 블로그 리뷰 수: 1286
블로그 탭: /hairshop/1142939643/review/ugc
15189970921481443590  준오헤어 노원사거리점차홍룸 용산점 크롤링을 진행합니다. 
크롤링을 진행합니다.
명시적 총 블로그 리뷰 수:명시적 총 블로그 리뷰 수:  3780973
블로그 탭:
 블로그 탭: /hairshop/1518997092/review/ugc
/hairshop/1481443590/review/ugc
1571970153 차홍룸 홍대점 크롤링을 진행합니다.
명시적 총 블로그 리뷰 수: 543
블로그 탭: /hairshop/1571970153/review/ugc
1177346002 차홍룸 양재점 크롤링을 진행합니다.
명시적 총 블로그 리뷰 수: 539
블로그 탭: /hairshop/1177346002/review/ugc
해당 엘리먼트를 찾지 못했습니다.
해당 엘리먼트를 찾지 못했습니다.


차홍룸 용산센트럴점의 블로그 리뷰 총 개수: 100

차홍룸 용산점의 블로그 리뷰 총 개수: 102
해당 엘리먼트를 찾지 못했습니다.



차홍룸 홍대점의 블로그 리뷰 총 개수: 100
해당 엘리먼트를 찾지 못했습니다.


...


박승철헤어스투디오 홍대점의 블로그 리뷰 총 개수: 100

제오헤어 이수점의 블로그 리뷰 총 개수: 100
해당 엘리먼트를 찾지 못했습니다.



그라티아의 블로그 리뷰 총 개수: 100
해당 엘리먼트를 찾지 못했습니다.
해당 엘리먼트를 찾지 못했습니다.



이철헤어커커 신림점의 블로그 리뷰 총 개수: 100



더 부띠끄 강남점의 블로그 리뷰 총 개수: 100


KeyboardInterrupt: 